In [10]:
### imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy
from torchvision import transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

import glob
import random

## Using MNSIT Dataset for Traning

In [ ]:
## Obtain data
X, y = fetch_openml("mnist_784", return_X_y=True, parser='auto')

In [ ]:
print(X.head())
print(y.head())

In [ ]:
## DataPreprocessing
_X = np.array(X).astype(np.float32)
_X = (_X > 150).astype(np.float32)
_y = np.array(y).astype(np.int64)
_X = _X.reshape([-1, 1, 28, 28])  ## CHW : Color channels, Height, Width 

print(_y)
print(type(_X))
print(_X.shape)
_y.dtype

In [ ]:
plt.imshow(_X[0].reshape((28, 28)))           

In [ ]:
## Train test Split
X_train, X_test, y_train, y_test = train_test_split(_X, _y, test_size=0.2, shuffle=True)

In [ ]:
print(X_train.shape)
y_train.shape
# plt.imshow(X_train[0])

In [ ]:
## Converting the data into a dataset for faster training
class Custom_MNIST_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [ ]:
## Default batch_size = 1
train_loader = DataLoader(Custom_MNIST_Dataset(X_train, y_train), batch_size=64)
test_loader = DataLoader(Custom_MNIST_Dataset(X_test, y_test), batch_size=64)

## Using Custom Made Dataset

In [4]:
class PrintedMNIST(Dataset):
    """Generates images containing a single digit from font"""

    def __init__(self, N, random_state, transform=None):
        """"""
        self.N = N
        self.random_state = random_state
        self.transform = transforms.ToTensor()
        fonts_folder = "../input/font-files"

        # self.fonts = ["Helvetica-Bold-Font.ttf", 'arial-bold.ttf']
        self.fonts = glob.glob(fonts_folder + "/*.ttf")

        # random.seed(random_state)

    def __len__(self):
        return self.N

    def __getitem__(self, idx):

        target = random.randint(2, 9)

        size = random.randint(150, 250)
        x = random.randint(30, 90)
        y = random.randint(30, 90)

        color = random.randint(200, 255)

        # Generate image
        img = Image.new("L", (256, 256))

        target = random.randint(2, 9)

        size = random.randint(150, 250)
        x = random.randint(30, 90)
        y = random.randint(30, 90)

        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype(random.choice(self.fonts), size)
        draw.text((x, y), str(target), color, font=font)  # , font=font, font=font

        img = img.resize((28, 28), Image.BILINEAR)
        if self.transform:
            img = self.transform(img)
            
        img = (img > 0.5).to(torch.float32)

        return img, target

In [5]:
train_loader = DataLoader(PrintedMNIST(60000, 1), batch_size=64)
test_loader = DataLoader(PrintedMNIST(6000, 3), batch_size=64)

In [6]:
data = PrintedMNIST(3, 1)
print(data.__getitem__(2))
# plt.imshow(data.__getitem__(0)[0])

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

## LeNet CNN

In [7]:
## Defing my Neural Network : LeNet
class MNIST_Classifier(nn.Module):
    def __init__(self):
        super(MNIST_Classifier, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        # self.softmax = nn.Softmax()
        
    def forward(self, input):
        x = self.conv1(input)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        # x = self.softmax(x)
        return x

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MNIST_Classifier().to(device)
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
metric = Accuracy(task="multiclass", num_classes=11).to(device)
epoches = 100;

In [ ]:
# ######### Loading data using zip : SLOW ##########

# for epoch in range(epoches):
#     count = 0
#     for img, label in zip(X_train, y_train):
#         img = img[None, :, :, :].to(device)
#         # print(img.shape)
        
#         output = model.forward(img)
#         count+= 1
#         if(count % 4000 == 0):
#             print(count)
#         target = label
#         # for i in range(11):
#         #     if(i == label): target.append(1)
#         #     else: target.append(0)
#         target = torch.tensor([target], dtype=torch.int64).to(device)
#         # target = target[None, :]
#         # print(target)
#         loss = loss_criterion(output, target)
#         loss.backward()  # computes gradients
#         optimizer.step() # updates weights
        
#         optimizer.zero_grad()
        
#     print(f"Epoch: {epoch + 1}, Loss: {loss.item():.4f}")


## Training Loop

In [11]:
for epoch in range(epoches):
    metric.reset()
    total_loss = 0;
    for img, label in train_loader:
        img = img.to(device, dtype= torch.float32)
        label = label.to(device, dtype= torch.int64)
        output = model.forward(img)

        loss = loss_criterion(output, label)
        total_loss = (total_loss + loss)/2
        # print(output)
        _, digit = torch.max(output, 1)

        metric.update(digit, label)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    accuracy = metric.compute()
    print(f"Epoch: {epoch + 1}, Loss: {total_loss.item():.4f}, Accuracy: {accuracy.item()}")   

Epoch: 1, Loss: 0.0045, Accuracy: 0.9882500171661377
Epoch: 2, Loss: 0.0051, Accuracy: 0.9960333108901978
Epoch: 3, Loss: 0.0025, Accuracy: 0.9973333477973938
Epoch: 4, Loss: 0.0021, Accuracy: 0.9974333047866821
Epoch: 5, Loss: 0.0022, Accuracy: 0.9988166689872742
Epoch: 6, Loss: 0.0005, Accuracy: 0.9987833499908447
Epoch: 7, Loss: 0.0002, Accuracy: 0.998449981212616
Epoch: 8, Loss: 0.0001, Accuracy: 0.9984166622161865
Epoch: 9, Loss: 0.0005, Accuracy: 0.9988833069801331
Epoch: 10, Loss: 0.0005, Accuracy: 0.9993500113487244
Epoch: 11, Loss: 0.0002, Accuracy: 0.9991999864578247
Epoch: 12, Loss: 0.0000, Accuracy: 0.9991166591644287
Epoch: 13, Loss: 0.0000, Accuracy: 0.9993000030517578
Epoch: 14, Loss: 0.0000, Accuracy: 1.0
Epoch: 15, Loss: 0.0001, Accuracy: 0.9990833401679993
Epoch: 16, Loss: 0.0000, Accuracy: 0.9998166561126709
Epoch: 17, Loss: 0.0001, Accuracy: 0.9994000196456909
Epoch: 18, Loss: 0.0002, Accuracy: 0.9993000030517578
Epoch: 19, Loss: 0.0001, Accuracy: 0.9997666478157043

KeyboardInterrupt: 

## Testing Loop

In [12]:
metric.reset()
total_loss = 0
model.eval()
for img, label in test_loader:
        img = img.to(device, dtype= torch.float32)
        label = label.to(device, dtype= torch.int64)
        output = model.forward(img)
        _, digit = torch.max(output, 1)

        loss = loss_criterion(output, label)
        total_loss = (total_loss + loss)/2
        metric.update(digit, label)
        # metric.update(output, label)
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
accuracy = metric.compute()
print(f"Loss: {total_loss.item():.4f}, Accuracy: {accuracy.item()}")   

Loss: 0.0010, Accuracy: 0.9988333582878113


## Saving the model

In [13]:
torch.save(model.state_dict(), "MNIST_Classifier.pth")